In [1]:
from collections import OrderedDict
import time
import re
import os

from pprint import pprint
from tqdm import tqdm
import logging

import json
from path import Path
import tempfile

import numpy as np
import pandas as pd

basedir=r'/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/'

# import makehuman-cmd

https://bitbucket.org/duststorm01/makehuman-commandline

In [2]:
# from .mh_helpers import clean, short_hash, clean_modifier

mhpath = Path(os.path.abspath("../vendor/makehuman-commandline/makehuman"))

#===============================================================================
# Import Makehuman resources, needs to be with makehuman dir as current dir
#===============================================================================

appcwd = os.path.abspath(os.curdir)
sys.path.append(mhpath)
sys.path.append(appcwd)
sys.path.append('.')

with mhpath:
    import makehuman
    oldpath = os.sys.path
    makehuman.set_sys_path()
    # make makehuman paths absolute by going through newest paths and making abs
    for i in range(len(os.sys.path)):
        p = os.sys.path[i]
        if p[0:2] == './':
            os.sys.path[i] = os.path.join(
                os.path.abspath('.'), p.replace('./', ''))
        else:
            break

    makehuman.init_logging()
    logging.getLogger().setLevel(logging.CRITICAL)
    #import image_pil as image_lib
    #
    import proxy as mhproxy
    import humanargparser
    import targets as mhtargets
    from human import Human
    import files3d
    import getpath
    import humanmodifier
    from core import G
    import headless
    import autoskinblender
    import export
    
    # Init console app
    with mhpath:
        G.app = headless.ConsoleApp()
#     G.app.selectedHuman = human = getHuman()
    headless.OBJExporter = None
    headless.MHXExporter = None
    headless.MhxConfig = None
    humanargparser.mods_loaded = False
    
    from makehuman import LicenseInfo
    mh_licence=LicenseInfo()


Initialized logging


# Export poses

We export the rotation matrix for each bone in each pose to json. Then they can be loaded into three.js with
```js
bone.setFromRotationMatrix(m)
```

In [5]:
import bvh
pose_files = list(getpath.search(['data/poses'], ['.bvh'], True))
pose_files

[u'data/poses/fly02.bvh',
 u'data/poses/fight01.bvh',
 u'data/poses/fight02.bvh',
 u'data/poses/fight03.bvh',
 u'data/poses/fight04.bvh',
 u'data/poses/fly01.bvh',
 u'data/poses/run01.bvh',
 u'data/poses/sit01.bvh',
 u'data/poses/standing01.bvh',
 u'data/poses/standing02.bvh',
 u'data/poses/standing03.bvh',
 u'data/poses/standing04.bvh',
 u'data/poses/standing05.bvh',
 u'data/poses/standing06.bvh',
 u'data/poses/tpose.bvh',
 u'data/poses/gym01.bvh',
 u'data/poses/mh-rigging351-benchmark1.bvh',
 u'data/poses/arms_akimbo/akimbo01.bvh',
 u'data/poses/prostrate/prostrate01.bvh',
 u'data/poses/sitting-pose/sitting.bvh',
 u'data/poses/sit_on_ground_01/sit_on_ground01.bvh',
 u'data/poses/snorkel_diver_01/Snorkel_Diver_01.bvh',
 u'data/poses/genuflect/genuflect01.bvh',
 u'data/poses/harvey_fashioncloseup2/Harvey_FashionCloseup2.bvh',
 u'data/poses/harvey_fashionclosup1/Harvey_FashionCloseup1.bvh',
 u'data/poses/harvey_fightpose1/Harvey_Fightpose1.bvh',
 u'data/poses/harvey_fightpose2/Harvey_Fi

In [13]:
# data={}
# for pose_file in pose_files:
#     bvh_obh = bvh.load(pose_file)
#     pose_data=data[bvh_obh.name]={}
#     for bone in bvh_obh.bvhJoints:
#         m=np.identity(4, dtype=np.float32)
#         m[:3]=bone.matrixPoses
#         pose_data[bone.name]=m.flatten().tolist()
    

# json.dump(data,open('/tmp/poses.json','w'))

In [5]:
# # # no doesn't work
# data={}
# pose_files = list(getpath.search(['data/poses'], ['.bvh'], True))
# for pose_file in pose_files:
#     bvh_obh = bvh.load(pose_file)
#     pose_data=data[bvh_obh.name]={}
#     for bone in bvh_obh.bvhJoints:
#         m=bone.matRestRelative
#         pose_data[bone.name]=m.flatten().tolist()
    

# json.dump(data,open('/tmp/poses_relative.json','w'))

In [6]:
# # no doesn't work
# data={}
# pose_files = list(getpath.search(['data/poses'], ['.bvh'], True))
# for pose_file in pose_files:
#     bvh_obh = bvh.load(pose_file)
#     pose_data=data[bvh_obh.name]={}
#     for bone in bvh_obh.bvhJoints:
#         m=bone.matRestGlobal
#         pose_data[bone.name]=m.flatten().tolist()
    

# json.dump(data,open('/tmp/poses_global.json','w'))

In [12]:
# export the normalised quarterions
import transformations as tm
# just the joints we need
frame=0
data={}
pose_files = list(getpath.search(['data/poses'], ['.bvh'], True))
for pose_file in pose_files:
    bvh_obh = bvh.load(pose_file)
    pose_data=data[bvh_obh.name]={}
    bones=[b.name for b in bvh_obh.bvhJoints]
    jointAnimList = [joint.name for joint in bvh_obh.getJoints() if not joint.isEndConnector()]
    for i, name in enumerate(jointAnimList):        
        bone = bvh_obh.bvhJoints[bones.index(name)]        
        assert bone.name == name
        
        
        quart = tm.quaternion_from_matrix(bone.matrixPoses[frame])
        
        quart = np.array([quart[1],quart[2],quart[3],quart[0]])
        if np.abs(1-(quart**2).sum())>0.001:
            print(name, (quart**2).sum())
        pose_data[name]=quart.tolist()

json.dump(data,open('/tmp/poses.json','w'))

In [28]:
os.sys.path.append(basedir)
from export_makehuman import copyAndCompress
outdir = Path(basedir).joinpath('..','public','data','poses')

In [33]:
# TODO copy thumbnail
for pose_file in pose_files:
    thumbnail = Path(pose_file.replace('.bvh','.thumb'))
    outnail = outdir.joinpath(Path(thumbnail).basename()).replace('.thumb','.thumb.png')
    
    # copy thumbnail
    if thumbnail.isfile():
        copyAndCompress(thumbnail,outnail)

# try setting poses then exporting - nope, same as exporting poses

In [8]:
# # make base human model with helpers enabled
# rigfile='data/rigs/default_no_toes.mhskel'

# def getHuman():
#     """Load a human model with modifiers."""
#     with mhpath:
#         # maxFaces *uint* Number of faces per vertex (pole), None for default (min 4)
#         human = Human(files3d.loadMesh(
#             getpath.getSysDataPath("3dobjs/base.obj"),
#             maxFaces=5))
#         # load modifiers onto human
#         humanmodifier.mods_loaded = False
#         modifiers = humanmodifier.loadModifiers(
#             getpath.getSysDataPath('modifiers/modeling_modifiers.json'), human)
#         return human

# with mhpath: 
#     basehuman = getHuman()
    
#     # rig
#     if rigfile:
#         humanargparser.addRig(basehuman,rigfile)    
    
#     mesh = basehuman.meshData
#     group_mask = np.ones(len(mesh._faceGroups), dtype=bool)
#     face_mask = group_mask[mesh.group]
#     basehuman._staticFaceMask = face_mask
#     basehuman.meshData.changeFaceMask(basehuman.staticFaceMask)
#     basehuman.meshData.updateIndexBufferFaces()
#     basehuman.changeVertexMask(None)

In [94]:
# data={}
# pose_files = list(getpath.search(['data/poses'], ['.bvh'], True))
# for pose_file in pose_files:
#     bvh_obh = bvh.load(pose_file)
#     pose_data=data[bvh_obh.name]={}
#     for bone in bvh_obh.bvhJoints:
#         m=np.identity(4, dtype=np.float32)
#         m[:3]=bone.matrixPoses
#         pose_data[bone.name]=m.flatten().tolist()
        
# data2={}
# for pose_name in data:
#     skeleton = basehuman.getSkeleton()
#     matPoses = np.array(data['fight01'].values()).reshape((-1,4,4))
#     skeleton.setPose(matPoses)
#     pose_data=data2[bvh_obh.name]={}
#     for bone in skeleton.bones.values():
#         m = bone.matPose
#         pose_data[bone.name]=m.flatten().tolist()

# json.dump(data,open('/tmp/poses_through_skel.json','w'))

# QC

In [14]:
# # export the normalised quarterions
# import transformations as tm


# # just the joints we need
# jointAnimList = [joint.name for joint in bvh_obh.getJoints() if not joint.isEndConnector()]

# frame=0
# data={}
# pose_files = list(getpath.search(['data/poses'], ['.bvh'], True))
# for pose_file in pose_files:
#     bvh_obh = bvh.load(pose_file)
#     pose_data=data[bvh_obh.name]={}
#     bones=[b.name for b in bvh_obh.bvhJoints]
#     for i, name in enumerate(jointAnimList):        
#         bone = bvh_obh.bvhJoints[bones.index(name)]
#         assert bone.name == name
# #         assert bone.rotOrder=='syzx', bone.rotOrder
# #         assert bone.rotOrder=='szyx', bone.rotOrder
        
#         quart = tm.quaternion_from_matrix(bone.matrixPoses[frame])
        
#         quart = np.array([quart[1],quart[2],quart[3],quart[0]])
#         quart=quart/quart.sum()
#         pose_data[name]=quart.tolist()

# json.dump(data,open('/tmp/poses_quart.json','w'))

# QC

I have a problem in many poses with the leg and elbow, these are the joins that add to less the 0.9 as below. I'm now sure why or how to fix.

Note rotOrder doesn't matter as it's adjusted for on load

In [408]:
# qc
bvh_obh = bvh.load(u'data/poses/standing06.bvh')
pose_data=data[bvh_obh.name]={}
bones=[b.name for b in bvh_obh.bvhJoints]
jointAnimList = [joint.name for joint in bvh_obh.getJoints() if not joint.isEndConnector()]
for i, name in enumerate(jointAnimList):        
    bone = bvh_obh.bvhJoints[bones.index(name)]    
    assert bone.name == name

    quart = tm.quaternion_from_matrix(bone.matrixPoses[frame])

    quart = np.array([quart[1],quart[2],quart[3],quart[0]])
    if quart.sum()<0.9:
        print(name, (quart**2).sum(), quart.sum(),np.around(quart,4).tolist())
    assert quart[3]>max(quart[0],quart[1],quart[2])
    pose_data[name]=quart.tolist()

('upperleg01.R', 0.99999999999999956, -0.069166761370897611, [-0.5243, -0.0902, -0.2603, 0.8057])
('upperleg02.R', 0.99999999999999989, 0.68513781362291948, [-0.165, -0.1418, 0.0161, 0.9759])
('lowerleg02.R', 0.99999999999999956, 0.79853554953842243, [-0.0152, -0.1661, -0.0061, 0.986])
('upperarm02.R', 1.0000000000000009, 0.63476654136876021, [-0.1595, -0.1798, 0.0034, 0.9707])
('lowerarm01.L', 1.0000000000000002, 0.4033462473791618, [-0.2115, -0.318, 0.0086, 0.9242])
('finger5-1.L', 1.0000000000000004, 0.72323297426417343, [0.0666, -0.0718, -0.2379, 0.9663])
('finger4-1.L', 1.0000000000000002, 0.88216344560402959, [0.0739, -0.0472, -0.1319, 0.9874])
('finger1-2.R', 0.99999999999999933, 0.84278011087238203, [-0.0276, -0.0945, -0.0298, 0.9947])
('finger3-2.L', 1.0000000000000004, 0.82669275135918396, [0.1483, -0.1184, -0.1702, 0.967])
('finger5-2.L', 1.0000000000000004, 0.79414246103043329, [0.0528, -0.0641, -0.1756, 0.981])
('finger4-2.L', 1.0000000000000002, 0.75807875653706835, [0.14